<a href="https://colab.research.google.com/github/eduardo-eal/TCC/blob/main/TCC1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#FAKE NEWS

Instalar bibliotecas

In [1]:
!pip install texthero
!pip install -U spacy

     |████████████████████████████████| 235 kB 7.1 MB/s 
     |████████████████████████████████| 1.5 MB 51.6 MB/s 
     |████████████████████████████████| 748 kB 33.9 MB/s 
  Attempting uninstall: regex
    Found existing installation: regex 2019.12.20
    Uninstalling regex-2019.12.20:
      Successfully uninstalled regex-2019.12.20
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5
     |████████████████████████████████| 6.0 MB 4.4 MB/s 
     |████████████████████████████████| 10.1 MB 14.9 MB/s 
     |████████████████████████████████| 628 kB 52.4 MB/s 
     |████████████████████████████████| 451 kB 49.6 MB/s 
     |████████████████████████████████| 181 kB 70.4 MB/s 
     |████████████████████████████████| 42 kB 1.4 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempti

Importar bibliotecas necessárias

In [2]:
import pandas as pd
import texthero as hero

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/usr/local/lib/python3.7/dist-packages/spacy/util.py:841: UserWarning: [W094] Model 'en_core_web_sm' (2.2.5) specifies an under-constrained spaCy version requirement: >=2.2.2. This can lead to compatibility problems with older versions, or as new spaCy versions are released, because the model may say it's compatible when it's not. Consider changing the "spacy_version" in your meta.json to a version range, with a lower and upper pin. For example: >=3.2.1,<3.3.0
  warnings.warn(warn_msg)


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


Importar Dataset de notícias verdadeiras e falsas

In [3]:
#arquivos podem ser obtidos em https://www.kaggle.com/clmentbisaillon/fake-and-real-news-dataset
noticias_verdadeiras = pd.read_csv('/content/drive/MyDrive/PUC/16 - Projeto Integrado – Inteligência Artificial e Aprendizado de Máquina (2021)/True.csv');
noticias_falsas = pd.read_csv('/content/drive/MyDrive/PUC/16 - Projeto Integrado – Inteligência Artificial e Aprendizado de Máquina (2021)/Fake.csv');

Quantidade de registros

In [33]:
noticias_verdadeiras.count()

title      21417
text       21417
subject    21417
date       21417
fake       21417
dtype: int64

In [32]:
noticias_falsas.count()

title      23481
text       23481
subject    23481
date       23481
fake       23481
dtype: int64

Primeiras notícias verdadeiras

In [4]:
noticias_verdadeiras.head(10)

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"
5,"White House, Congress prepare for talks on spe...","WEST PALM BEACH, Fla./WASHINGTON (Reuters) - T...",politicsNews,"December 29, 2017"
6,"Trump says Russia probe will be fair, but time...","WEST PALM BEACH, Fla (Reuters) - President Don...",politicsNews,"December 29, 2017"
7,Factbox: Trump on Twitter (Dec 29) - Approval ...,The following statements were posted to the ve...,politicsNews,"December 29, 2017"
8,Trump on Twitter (Dec 28) - Global Warming,The following statements were posted to the ve...,politicsNews,"December 29, 2017"
9,Alabama official to certify Senator-elect Jone...,WASHINGTON (Reuters) - Alabama Secretary of St...,politicsNews,"December 28, 2017"


In [5]:
noticias_verdadeiras.loc[7,'text']

'The following statements\xa0were posted to the verified Twitter accounts of U.S. President Donald Trump, @realDonaldTrump and @POTUS.  The opinions expressed are his own.\xa0Reuters has not edited the statements or confirmed their accuracy.  @realDonaldTrump : - While the Fake News loves to talk about my so-called low approval rating, @foxandfriends just showed that my rating on Dec. 28, 2017, was approximately the same as President Obama on Dec. 28, 2009, which was 47%...and this despite massive negative Trump coverage & Russia hoax! [0746 EST] - Why is the United States Post Office, which is losing many billions of dollars a year, while charging Amazon and others so little to deliver their packages, making Amazon richer and the Post Office dumber and poorer? Should be charging MUCH MORE! [0804 EST] -- Source link: (bit.ly/2jBh4LU) (bit.ly/2jpEXYR) '

Primeiras notícias falsas

In [6]:
noticias_falsas.head(10)

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017"
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017"
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017"
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017"
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017"


Em uma primeira análise já é possível identificar um padrão no início dos textos das notícias verdadeiras, com o nome da cidade seguido de "(Reuters)" que não é observado nas falsas, provavelmente devido à diferença de fontes das quais foram extraídas. Teremos que tratar/remover isso para não gerar um viés para as notícias verdadeiras.

Vamos criar uma nova coluna com a classe e juntar as bases para fazer um pré-processamento para todo o Dataset

In [7]:
noticias_verdadeiras['fake']=0
noticias_falsas['fake']=1
noticias = pd.concat([noticias_verdadeiras,noticias_falsas])

In [8]:
noticias.head()

,title,text,subject,date,fake
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",0


Embaralhar as notícias verdadeiras e falsas

In [9]:
noticias = noticias.sample(frac=1.0).reset_index(drop=True)

noticias.head()

,title,text,subject,date,fake
0,LOL! HOW TO TRIGGER A LIBERAL On Halloween: “M...,It probably explains why we love this guy so m...,left-news,"Oct 27, 2016",1
1,DEMOCRAT MAXINE WATERS GOES OFF THE RAILS: Sme...,This Congresswoman has a potty mouth and a bit...,politics,"May 10, 2017",1
2,Iran president defends Guards in show of unity...,BEIRUT (Reuters) - Iran s moderate president g...,worldnews,"October 11, 2017",0
3,"Michael Flynn Begs For Donations On Twitter, ...",Remember that time Michael Flynn led lock her...,News,"September 18, 2017",1
4,Colombia protests what it says was Venezuelan ...,BOGOTA (Reuters) - Colombia has sent a letter ...,worldnews,"August 28, 2017",0


In [10]:
noticias.loc[2,'text']

'BEIRUT (Reuters) - Iran s moderate president gave a full-throated defense of his one-time rivals in the Revolutionary Guards on Wednesday, as the country s pragmatist and hardline factions rallied together in the face of threats from Donald Trump. The U.S. president is expected to  decertify  Iran s nuclear deal with global powers this week and add its Revolutionary Guards military force to Washington s black list of terrorist groups under a strategy to increase pressure on Tehran. The threat of U.S. action has united the two main factions of Iran s leadership, with the pragmatists led by President Hassan Rouhani who seek greater openness to the West demonstrating their support for the hardline Guards. During a cabinet meeting shown on state television on Wednesday, Rouhani said U.S. action against the Guards would be a  mistake beyond mistakes .  They think that the Guards are a military entity. The Revolutionary Guards are not a military entity. They re in the heart of the people. T

Verificar campos nulos - Nenhum campo nulo no dataset

In [11]:
noticias.isnull().sum()

title      0
text       0
subject    0
date       0
fake       0
dtype: int64

#Pré-processamento do título e texto

Vamos usar a biblioteca texthero para fazer uma limpeza no texto, retirando caracteres sem importância para a identificação de padrões e assim deixar o texto mais significativo para os algoritmos de IA.

In [12]:
#Alterar todas as palavras para minúsculas
noticias['text'] = hero.lowercase(noticias['text'])

noticias['title'] = hero.lowercase(noticias['title'])

#Remover pontuação
noticias['text'] = hero.remove_punctuation(noticias['text'])

noticias['title'] = hero.remove_punctuation(noticias['title'])

#Remover acentos
noticias['text'] = hero.remove_diacritics(noticias['text'])

noticias['title'] = hero.remove_diacritics(noticias['title'])

#Remover dígitos
noticias['text'] = hero.remove_digits(noticias['text'])

noticias['title'] = hero.remove_digits(noticias['title'])


#Remover chaves, parênteses, colchetes...
noticias['text'] = hero.remove_brackets(noticias['text'])
noticias['text'] = hero.remove_angle_brackets(noticias['text'])
noticias['text'] = hero.remove_curly_brackets(noticias['text'])
noticias['text'] = hero.remove_round_brackets(noticias['text'])
noticias['text'] = hero.remove_square_brackets(noticias['text'])

noticias['title'] = hero.remove_brackets(noticias['title'])
noticias['title'] = hero.remove_angle_brackets(noticias['title'])
noticias['title'] = hero.remove_curly_brackets(noticias['title'])
noticias['title'] = hero.remove_round_brackets(noticias['title'])
noticias['title'] = hero.remove_square_brackets(noticias['title'])

#remover stopwords
noticias['text'] = hero.remove_stopwords(noticias['text'])

noticias['title'] = hero.remove_stopwords(noticias['title'])

#remover URL
noticias['text'] = hero.remove_urls(noticias['text'])

noticias['title'] = hero.remove_urls(noticias['title'])

#remover espaços em branco
noticias['text'] = hero.remove_whitespace(noticias['text'])

noticias['title'] = hero.remove_whitespace(noticias['title'])




Remover do título e texto palavas com menos de 3 caracteres e termo reuters (presente na maioria das notícias verdadeiras)

In [13]:
noticias['text'] = noticias['text'].apply(lambda x: ' '.join([termo for termo in x.split() if len(termo) > 2 and termo != 'reuters']))
noticias['title'] = noticias['title'].apply(lambda x: ' '.join([termo for termo in x.split() if len(termo) > 2 and termo != 'reuters']))

In [14]:
hero.top_words(noticias['text'])[:20]

trump         134052
said          132825
president      55893
would          55167
people         41857
one            37927
state          34488
also           31357
new            31311
clinton        28695
obama          28203
donald         28137
government     28048
house          27752
states         26845
republican     25568
year           24998
could          23957
united         23601
told           23367
Name: text, dtype: int64

Após limpeza do texto, ainda não existem campos nulos, mas o texto de 632 registros ficaram vazios

In [15]:
noticias.isnull().sum()

title      0
text       0
subject    0
date       0
fake       0
dtype: int64

In [16]:
#Textos vazios
(noticias['text'] == '').sum()

632

In [17]:
noticias_sem_texto = noticias[noticias['text'] == '']

In [18]:
noticias_sem_texto.groupby('fake').count()

,title,text,subject,date
fake,,,,
0,1,1,1,1
1,631,631,631,631


In [19]:
#Títulos vazios - nenhum registro
(noticias['title'] == '').sum()

0

In [20]:
#Título das notícias verdadeiras sem texto
noticias_sem_texto[noticias_sem_texto['fake']==0]['title']

26165    graphic supreme court roundup
Name: title, dtype: object

In [21]:
#Título das notícias falsas sem texto
noticias_sem_texto[noticias_sem_texto['fake']==1]['title']

24       whoa angry obama berates black caucus rant 'le...
116      clueless nevada poll workers break law wearing...
146                          best tweet week "hillary sale
173      liberals see light huffpo columnist lets rip o...
338      video judge napolitano fills order "rolling re...
                               ...                        
44512    judge jeanine gives great debate advice trump ...
44554    new evidence proves hillary manipulated senate...
44584    say nancy pelosi spits word salad press confer...
44607    horrible end game bernie sanders calls sociali...
44641    epic judge jeanine pirro rips michelle obama r...
Name: title, Length: 631, dtype: object

Como existem 632 registros com texto em branco mas com o título contendo informação significativa, vamos concatenar título e texto para tratar esses dois campos como um único.

In [22]:
#campo noticia_tratada vai conter o título seguido do texto
noticias['noticia_tratada'] = noticias['title']+' '+noticias['text']

In [23]:
noticias.head()

,title,text,subject,date,fake,noticia_tratada
0,"lol trigger liberal halloween ""make costume ta...",probably explains love guy much strong languag...,left-news,"Oct 27, 2016",1,"lol trigger liberal halloween ""make costume ta..."
1,democrat maxine waters goes rails smears trump...,congresswoman potty mouth bitter attitude towa...,politics,"May 10, 2017",1,democrat maxine waters goes rails smears trump...
2,iran president defends guards show unity antic...,beirut iran moderate president gave full throa...,worldnews,"October 11, 2017",0,iran president defends guards show unity antic...
3,michael flynn begs donations twitter gets wrec...,remember time michael flynn led lock chants ra...,News,"September 18, 2017",1,michael flynn begs donations twitter gets wrec...
4,colombia protests says venezuelan military inc...,bogota colombia sent letter protest venezuela ...,worldnews,"August 28, 2017",0,colombia protests says venezuelan military inc...


##Tokenizar e fazer lemmatization
Para preparar o texto para uso nos algoritmos de IA temos que tokenizar (dividir o texto em palavras) e em seguida vamos aplicar o lemmatization. Optamos por usar Lemmatization e não stemming pois lemmatization reduz as palavras a uma ainda existente na linguagem. O Stemming reduz a palavra a uma raiz não necessariamente uma palavra válida na linguagem, o que poderia dificultar nossa análise nos próximos passos.

In [24]:
#tokenização e lemmatização com wordnet está rápida (pouco mais de 1min) mas deixando muitas palavras como verbos e conjugações diferentes
#import nltk
#nltk.download('wordnet')
#nltk.download('punkt')
#nltk.download('omw-1.4')

#lemmatizer = nltk.stem.WordNetLemmatizer()

#def f_lemmatize(texto):
#    return [lemmatizer.lemmatize(termo) for termo in nltk.word_tokenize(texto)]

#noticias['noticia_tratada_lemma'] = noticias['noticia_tratada'].apply(f_lemmatize)

In [25]:
#tokenização e lematização com spacy está levando 30 min mas o resultado está melhor processado que com Wordnet
import spacy
nlp = spacy.load('en_core_web_sm')

def f_lemmatize(texto):
    return [termo.lemma_ for termo in nlp(texto)]

noticias['noticia_tratada_lemma'] = noticias['noticia_tratada'].apply(f_lemmatize)

In [26]:
noticias.head()

,title,text,subject,date,fake,noticia_tratada,noticia_tratada_lemma
0,"lol trigger liberal halloween ""make costume ta...",probably explains love guy much strong languag...,left-news,"Oct 27, 2016",1,"lol trigger liberal halloween ""make costume ta...","[lol, trigger, liberal, halloween, "", make, co..."
1,democrat maxine waters goes rails smears trump...,congresswoman potty mouth bitter attitude towa...,politics,"May 10, 2017",1,democrat maxine waters goes rails smears trump...,"[democrat, maxine, water, go, rail, smear, tru..."
2,iran president defends guards show unity antic...,beirut iran moderate president gave full throa...,worldnews,"October 11, 2017",0,iran president defends guards show unity antic...,"[iran, president, defend, guard, show, unity, ..."
3,michael flynn begs donations twitter gets wrec...,remember time michael flynn led lock chants ra...,News,"September 18, 2017",1,michael flynn begs donations twitter gets wrec...,"[michael, flynn, begs, donation, twitter, get,..."
4,colombia protests says venezuelan military inc...,bogota colombia sent letter protest venezuela ...,worldnews,"August 28, 2017",0,colombia protests says venezuelan military inc...,"[colombia, protest, say, venezuelan, military,..."


In [27]:
noticias.drop(columns=['noticia_tratada'], inplace=True)


In [28]:
#Gravar o dataset processado até o momento
noticias.to_csv('/content/drive/MyDrive/PUC/16 - Projeto Integrado – Inteligência Artificial e Aprendizado de Máquina (2021)/noticias_preprocessadas.csv', index=False)
#Arquivo disponível em https://drive.google.com/file/d/1GDt17BwSrxJF5WJ0F8c0il6nO8xD7wCh/view?usp=sharing